In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1607357564198_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
from datetime import date
today = date.today()

day = today.strftime("%d-%m-%Y")

datos = spark.read.csv('s3://proyecto-bigdata/raw/datos-{0}.csv'.format(day), inferSchema=True, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
datos.show(10)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----+------------------------+----+----------------+------------------+------+-------------------+--------------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+------------------+-----------------------+
|fecha reporte web|ID de caso|Fecha de notificación|Código DIVIPOLA departamento|Nombre departamento|Código DIVIPOLA municipio|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|Código ISO del país|     Nombre del país|Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|Pertenencia étnica|Nombre del grupo étnico|
+-----------------+----------+---------------------+----------------------------+-------------------+-------------------------+----------------+----

In [4]:
datos_limpios = datos.drop('Código DIVIPOLA municipio','Código DIVIPOLA departamento','Unidad de medida de edad','Código ISO del país','Pertenencia étnica','Nombre del grupo étnico')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
def edad_etapa(edad):
    etapa = ''
    if (edad<=5):
        etapa = 'primera infancia'
    elif (5<edad<=11):
        etapa = 'infancia'
    elif (11<edad<=18):
        etapa = 'adolescencia'
    elif (18<edad<=26):
        etapa = 'juventud'
    elif (26<edad<=59):
        etapa = 'adultez'
    elif (edad>=60):
        etapa = 'vejez'
    return etapa

#Agregar Columna
etapa_udf = spark.udf.register('Etapas', edad_etapa)

datos_limpios = datos_limpios.withColumn('Etapa', etapa_udf(datos['Edad']))

datos_limpios.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+-------------------+----------------+----+----+----------------+------------------+------+--------------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+--------+
|fecha reporte web|ID de caso|Fecha de notificación|Nombre departamento|Nombre municipio|Edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|     Nombre del país|Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|   Etapa|
+-----------------+----------+---------------------+-------------------+----------------+----+----+----------------+------------------+------+--------------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+--------+
| 6/3/2020 0:00:00|         1|     2/3/2020 0:00:00|             BOGOTA|          BOGOTA|  19|   F|       Importado|        

In [6]:
datos_limpios = datos_limpios.fillna('COLOMBIA', subset=['Nombre del país'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
datos_limpios.select('Nombre del país').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|     Nombre del país|
+--------------------+
|              ITALIA|
|              ESPAÑA|
|              ESPAÑA|
|            COLOMBIA|
|            COLOMBIA|
|            COLOMBIA|
|ESTADOS UNIDOS DE...|
|              ESPAÑA|
|              ESPAÑA|
|              ESPAÑA|
|              ESPAÑA|
|              ITALIA|
|            COLOMBIA|
|              ESPAÑA|
|              ESPAÑA|
|              ITALIA|
|              ESPAÑA|
|              ESPAÑA|
|            COLOMBIA|
|            COLOMBIA|
+--------------------+
only showing top 20 rows

In [8]:
datos_limpios.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+----------+---------------------+-------------------+----------------+----+----+----------------+------------------+------+---------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+--------+
|fecha reporte web|ID de caso|Fecha de notificación|Nombre departamento|Nombre municipio|Edad|Sexo|Tipo de contagio|Ubicación del caso|Estado|Nombre del país|Recuperado|Fecha de inicio de síntomas|Fecha de muerte|Fecha de diagnóstico|Fecha de recuperación|Tipo de recuperación|   Etapa|
+-----------------+----------+---------------------+-------------------+----------------+----+----+----------------+------------------+------+---------------+----------+---------------------------+---------------+--------------------+---------------------+--------------------+--------+
| 6/3/2020 0:00:00|         1|     2/3/2020 0:00:00|             BOGOTA|          BOGOTA|  19|   F|       Importado|              Casa|  Le

In [9]:
from datetime import date

today = date.today()

d4 = today.strftime("%m-%d-%Y")

url = 's3://proyecto-bigdata/curated/{0}'.format(d4)

datos_limpios.coalesce(1).write.format('csv').option('header','True').save(url)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
paises = datos_limpios.groupBy('Nombre del país').count().orderBy('count', ascending=False)

paises = paises.withColumnRenamed("Nombre del país","pais_donde_viajo")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
paises.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+
|    pais_donde_viajo|  count|
+--------------------+-------+
|            COLOMBIA|1370152|
|              ESPAÑA|    258|
|ESTADOS UNIDOS DE...|    223|
|             ECUADOR|     68|
|              BRASIL|     60|
|              MÉXICO|     55|
|              PANAMA|     34|
|             TURQUÍA|     27|
|               CHILE|     20|
|             FRANCIA|     19|
|                PERU|     18|
|           VENEZUELA|     18|
|             JAMAICA|     16|
|REINO UNIDO DE GR...|     14|
|         PUERTO RICO|     13|
|              EGIPTO|     11|
|              ITALIA|     11|
|            ALEMANIA|     11|
|REPÚBLICA DOMINICANA|     10|
|ANTILLAS NEERLAND...|      8|
+--------------------+-------+
only showing top 20 rows

In [12]:
recuperados_y_fallecidos_departamento = datos_limpios.groupBy('Nombre departamento','Recuperado').count().orderBy('count', ascending=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
recuperados_y_fallecidos_departamento.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------+------+
|Nombre departamento|Recuperado| count|
+-------------------+----------+------+
|             BOGOTA|Recuperado|358054|
|          ANTIOQUIA|Recuperado|205012|
|              VALLE|Recuperado|102582|
|       CUNDINAMARCA|Recuperado| 50601|
|          SANTANDER|Recuperado| 50139|
|       BARRANQUILLA|Recuperado| 43570|
|          ATLANTICO|Recuperado| 31289|
|          CARTAGENA|Recuperado| 31255|
|              HUILA|Recuperado| 29166|
|             TOLIMA|Recuperado| 29162|
|              CESAR|Recuperado| 27823|
|               META|Recuperado| 27605|
|    NORTE SANTANDER|Recuperado| 27551|
|            CORDOBA|Recuperado| 25847|
|             CALDAS|Recuperado| 23981|
|             NARIÑO|Recuperado| 23977|
|          RISARALDA|Recuperado| 22475|
|             BOGOTA|    Activo| 21540|
|             BOYACA|Recuperado| 20923|
|            QUINDIO|Recuperado| 16559|
+-------------------+----------+------+
only showing top 20 rows

In [14]:
recuperados_y_fallecidos_etapa = datos_limpios.groupBy('Etapa','Recuperado').count().orderBy('count', ascending=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
recuperados_y_fallecidos_etapa.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+----------+------+
|           Etapa|Recuperado| count|
+----------------+----------+------+
|         adultez|Recuperado|784073|
|        juventud|Recuperado|203227|
|           vejez|Recuperado|160739|
|    adolescencia|Recuperado| 56021|
|         adultez|    Activo| 41044|
|        infancia|Recuperado| 31450|
|           vejez| Fallecido| 28926|
|primera infancia|Recuperado| 21900|
|           vejez|    Activo| 15501|
|        juventud|    Activo| 10177|
|         adultez| Fallecido|  8243|
|    adolescencia|    Activo|  2963|
|           vejez|       N/A|  2287|
|        infancia|    Activo|  1514|
|primera infancia|    Activo|  1453|
|         adultez|       N/A|   736|
|        juventud| Fallecido|   244|
|           vejez| fallecido|   230|
|        juventud|       N/A|   111|
|         adultez| fallecido|    65|
+----------------+----------+------+
only showing top 20 rows

In [16]:
sexo = datos_limpios.groupBy('Sexo').count().orderBy('count', ascending=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
sexo.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+
|Sexo| count|
+----+------+
|   F|689057|
|   M|682046|
+----+------+

In [18]:
sexo_recuperado = datos_limpios.groupBy('Sexo','Recuperado').count().orderBy('count', ascending=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
sexo_recuperado.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+----------+------+
|Sexo|Recuperado| count|
+----+----------+------+
|   F|Recuperado|637847|
|   M|Recuperado|619563|
|   M|    Activo| 36401|
|   F|    Activo| 36251|
|   M| Fallecido| 24024|
|   F| Fallecido| 13488|
|   M|       N/A|  1861|
|   F|       N/A|  1372|
|   M| fallecido|   197|
|   F| fallecido|    99|
+----+----------+------+

In [20]:
recuperados_muertos_medellin = datos_limpios.filter(datos_limpios['Nombre municipio'] == 'MEDELLIN').groupBy('Nombre municipio', 'recuperado').count().orderBy('count', ascending=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
recuperados_muertos_medellin.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+----------+------+
|Nombre municipio|recuperado| count|
+----------------+----------+------+
|        MEDELLIN|Recuperado|117600|
|        MEDELLIN|    Activo|  6290|
|        MEDELLIN| Fallecido|  2291|
|        MEDELLIN|       N/A|   183|
|        MEDELLIN| fallecido|    12|
+----------------+----------+------+

In [22]:
from datetime import date

today = date.today()

d4 = today.strftime("%m-%d-%Y")

url_1 = 's3://proyecto-bigdata/refined/{0}/{1}'.format(d4, 'paises')
url_2 = 's3://proyecto-bigdata/refined/{0}/{1}'.format(d4, 'recuperados_y_fallecidos_departamento')
url_3 = 's3://proyecto-bigdata/refined/{0}/{1}'.format(d4, 'recuperados_y_fallecidos_etapa')
url_4 = 's3://proyecto-bigdata/refined/{0}/{1}'.format(d4, 'sexo')
url_5 = 's3://proyecto-bigdata/refined/{0}/{1}'.format(d4, 'recuperados_muertos_medellin')

paises.coalesce(1).write.format('csv').option('header','True').save(url_1)
recuperados_y_fallecidos_departamento.coalesce(1).write.format('csv').option('header','True').save(url_2)
recuperados_y_fallecidos_etapa.coalesce(1).write.format('csv').option('header','True').save(url_3)
sexo.coalesce(1).write.format('csv').option('header','True').save(url_4)
recuperados_muertos_medellin.coalesce(1).write.format('csv').option('header','True').save(url_5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+
|     Nombre del país|
+--------------------+
|              ITALIA|
|              ESPAÑA|
|              ESPAÑA|
|                null|
|                null|
|                null|
|ESTADOS UNIDOS DE...|
|              ESPAÑA|
|              ESPAÑA|
|              ESPAÑA|
|              ESPAÑA|
|              ITALIA|
|                null|
|              ESPAÑA|
|              ESPAÑA|
|              ITALIA|
|              ESPAÑA|
|              ESPAÑA|
|                null|
|                null|
+--------------------+
only showing top 20 rows